# 0. DEPENDENCIES

Fix for Jupyter Notebook imports:

In [89]:
import os
import sys

print(os.getcwd())

sys.path.append("S:\Dropbox\\000 - CARND\CarND-T1-P5-Vehicle-Detection")

for path in sys.path: print(path)

S:\Dropbox\000 - CARND\CarND-T1-P5-Vehicle-Detection\src\notebooks

C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\python35.zip
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\DLLs
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\cycler-0.10.0-py3.5.egg
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\IPython\extensions
C:\Users\gmzco\.ipython
S:\Dropbox\000 - CARND\CarND-T1-P5-Vehicle-Detection


Remove the additional entry if needed:

In [88]:
sys.path = sys.path[:-1]

for path in sys.path: print(path)


C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\python35.zip
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\DLLs
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\cycler-0.10.0-py3.5.egg
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\IPython\extensions
C:\Users\gmzco\.ipython


Load all dependencies:

In [114]:

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import pickle

from importlib import reload
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split

import src.helpers.constants as C
import src.helpers.io as IO
import src.helpers.features as FT


reload(C)
reload(IO)
reload(FT)


<module 'src.helpers.features' from 'S:\\Dropbox\\000 - CARND\\CarND-T1-P5-Vehicle-Detection\\src\\helpers\\features.py'>

# 1. LOAD DATA

First, load all images filenames on the datasets, split into cars and non-cars. Print the counts and percentages of each to verify that the dataset is balanced.

In [94]:

files_cars = glob.glob("../../input/images/dataset/vehicles/*/*.png")
files_no_cars = glob.glob("../../input/images/dataset/non-vehicles/*/*.png")

count_cars = len(files_cars)
count_no_cars = len(files_no_cars)
count_total = count_cars + count_no_cars

percent_cars = 100 * count_cars / count_total
percent_no_cars = 100 * count_no_cars / count_total

print("    CAR IMAGES {0:5d} = {1:6.2f} %".format(count_cars, percent_cars))
print("NON-CAR IMAGES {0:5d} = {1:6.2f} %".format(count_no_cars, percent_no_cars))
print("-------------------------------")
print("         TOTAL {0:5d} = 100.00 %".format(count_total))


    CAR IMAGES  8792 =  49.50 %
NON-CAR IMAGES  8968 =  50.50 %
-------------------------------
         TOTAL 17760 = 100.00 %


The data looks quite balanced, so no need to do any augmentation. 

Next, preload them and check their total size to see if it's feasible to preload them all in different color spaces.

In [100]:

# Load all images (RGB only):
imgs_cars = IO.load_images_rgb(files_cars)
imgs_no_cars = IO.load_images_rgb(files_no_cars)

# Calculate their size by dumping them:
size_cars_b = sys.getsizeof(pickle.dumps(imgs_cars))
size_no_cars_b = sys.getsizeof(pickle.dumps(imgs_no_cars))
size_total_b = size_cars_b + size_no_cars_b


Print results in multiple units and calculate total for all channels:

In [103]:

size_cars_mb = size_cars_b / 1048576
size_no_cars_mb = size_no_cars_b / 1048576
size_total_mb = size_total_b / 1048576
size_all_spaces_mb = size_total_mb * (1 + len(C.COLOR_SPACES)) # RGB not included in C.COLOR_SPACES

size_all_spaces_gb = size_all_spaces_mb / 1024

print("          CAR IMAGES SIZE = {0:12.2f} B  = {1:6.2f} MB".format(size_cars_b, size_cars_mb))
print("      NON-CAR IMAGES SIZE = {0:12.2f} B  = {1:6.2f} MB".format(size_no_cars_b, size_no_cars_mb))
print("---------------------------------------------------")
print("               TOTAL SIZE = {0:12.2f} B  = {1:6.2f} MB".format(size_total_b, size_total_mb))
print("ESTIMATED ALL SPACES SIZE = {0:12.2f} MB = {1:6.2f} GB".format(size_all_spaces_mb, size_all_spaces_gb))


          CAR IMAGES SIZE = 432680131.00 B  = 412.64 MB
      NON-CAR IMAGES SIZE = 441341619.00 B  = 420.90 MB
---------------------------------------------------
               TOTAL SIZE = 874021750.00 B  = 833.53 MB
ESTIMATED ALL SPACES SIZE =      5001.19 MB =   4.88 GB


Free up space:

In [104]:

try: del imgs_cars
except NameError: pass # Was not defined
    
try: del imgs_no_cars
except NameError: pass # Was not defined


Load all images in all color spaces:

In [125]:

# CARS:

imgs_cars_rgb, \
imgs_cars_hsv, \
imgs_cars_luv, \
imgs_cars_hls, \
imgs_cars_yuv, \
imgs_cars_ycrcb, \
imgs_cars_gray = IO.load_images_all(files_cars)

# NON-CARS:

imgs_no_cars_rgb, \
imgs_no_cars_hsv, \
imgs_no_cars_luv, \
imgs_no_cars_hls, \
imgs_no_cars_yuv, \
imgs_no_cars_ycrcb, \
imgs_no_cars_gray = IO.load_images_all(files_no_cars)


Some basic checks:

In [126]:

# CARS:

assert len(imgs_cars_rgb) == count_cars
assert len(imgs_cars_hsv) == count_cars
assert len(imgs_cars_luv) == count_cars
assert len(imgs_cars_hls) == count_cars
assert len(imgs_cars_yuv) == count_cars
assert len(imgs_cars_ycrcb) == count_cars
assert len(imgs_cars_gray) == count_cars

# NON-CARS:

assert len(imgs_no_cars_rgb) == count_no_cars
assert len(imgs_no_cars_hsv) == count_no_cars
assert len(imgs_no_cars_luv) == count_no_cars
assert len(imgs_no_cars_hls) == count_no_cars
assert len(imgs_no_cars_yuv) == count_no_cars
assert len(imgs_no_cars_ycrcb) == count_no_cars
assert len(imgs_no_cars_gray) == count_no_cars


Free up space:

In [123]:

# CARS:

try: del imgs_cars_rgb
except NameError: pass # Was not defined

try: del imgs_cars_hsv
except NameError: pass # Was not defined

try: del imgs_cars_luv
except NameError: pass # Was not defined

try: del imgs_cars_hls
except NameError: pass # Was not defined

try: del imgs_cars_yuv
except NameError: pass # Was not defined

try: del imgs_cars_ycrcb
except NameError: pass # Was not defined

try: del imgs_cars_gray
except NameError: pass # Was not defined

# NON-CARS:

try: del imgs_no_cars_rgb
except NameError: pass # Was not defined

try: del imgs_no_cars_hsv
except NameError: pass # Was not defined

try: del imgs_no_cars_luv
except NameError: pass # Was not defined

try: del imgs_no_cars_hls
except NameError: pass # Was not defined

try: del imgs_no_cars_yuv
except NameError: pass # Was not defined

try: del imgs_no_cars_ycrcb
except NameError: pass # Was not defined

try: del imgs_no_cars_gray
except NameError: pass # Was not defined


## SECTION'S CONCERNS, IMPROVEMENTS, TODOS...

- Should images that belong to the same sequence be grouped together so that half of each of them can go to a different subset (training and test)?

# 2. 

In [8]:


# Reduce the sample size because HOG features are slow to compute
# The quiz evaluator times out after 13s of CPU time
sample_size = 500
cars = cars[0:sample_size]
notcars = notcars[0:sample_size]

### TODO: Tweak these parameters and see how the results change.
colorspace = 'HSV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 2 # Can be 0, 1, 2, or "ALL"

# ACC = 0.890000 using HSV[0] 9 orientations, 8 ppc, 2 cpb
# ACC = 0.845000 using HSV[1] 9 orientations, 8 ppc, 2 cpb
# ACC = 0.925000 using HSV[2] 9 orientations, 8 ppc, 2 cpb

t=time.time()
car_features = extract_features(cars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = extract_features(notcars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

# Check the score of the SVC

print("ACC = %f using %s[%s] %d orientations, %d ppc, %d cpb" % (
    round(svc.score(X_test, y_test), 4),
    colorspace,
    str(hog_channel),
    orient,
    pix_per_cell,
    cell_per_block
))

# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

0.0 Seconds to extract HOG features...


ValueError: Found array with 0 feature(s) (shape=(2, 0)) while a minimum of 1 is required by StandardScaler.